# FunOS module dependency graph generator

In [ ]:
import argparse
import json

import numpy as np

import os
import sys

%matplotlib inline

import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 22})

#https://stackoverflow.com/questions/36288670/how-to-programmatically-generate-markdown-output-in-jupyter-notebooks
from IPython.display import display, Markdown, Latex

In [ ]:
# module depenency source file can be obtained from dpcsh output
# save the output from the following command to 'module_dep.json'
# dpcsh > debug module_dep


# *NOTE*: generate html with batch method
# $ python ./convert_nb.py --filename gen_module_dep.ipynb --execute

# *NOTE*: run this command to clean output cell and meta data.
# $ nb-clean clean ./gen_module_dep.ipynb



In [ ]:
# set this True to generate figures
png_gen = True

In [ ]:
try:
	# for table
	from tabulate import tabulate
	import networkx as nx
	import pandas as pd
except ImportError:
	print("{}: Import failed!".format(__file__))
	print("Install missing modules")
	print(">>> pip install tabulate pandas")
	sys.exit()

In [ ]:
# expecting module_dep.json is FunOS directory
module_file = "module_dep.json"

In [ ]:
with open(module_file, 'r') as f:
	module_dep = json.load(f)

# df = pd.DataFrame.from_dict(module_dep, orient='index')
# print(df)

In [ ]:
graph = nx.DiGraph()

In [ ]:
DEBUG = False

In [ ]:
# add module denepdency in graphx format

roots = set()
for i, (k, v) in enumerate(module_dep.items()):
    if DEBUG and i > 3:
        break
    if len(v) == 0:
        roots.add(k)
    else:
        for el in v:
            graph.add_edge(k, el)

In [ ]:
options = {
    'node_color': 'orange',
    'with_labels': True,
    'arrows': True,
    'node_size': 10,
    'width': 1,
}

## Dependency summaries

In [ ]:
# show table of modules


# table_str = tabulate(ol_sel], headers=headers, tablefmt="psql", floatfmt=".2f")
# shrink to 10 dependencies
# module_dep1 = {k:[v[] for k,v in module_dep.items()}

DEP_LIMIT = 5
# module_dep1 = {k:[v[:10]] for k,v in module_dep.items()}
headers=["module", "dependencies", "no. dep.", "truncated"]

module_dep1 = {}
for k, v in module_dep.items():
	if len(v) < DEP_LIMIT:
		module_dep1[k] = [v, "{}".format(len(v))]
	else:
		module_dep1[k] = [v[:DEP_LIMIT], "{}".format(len(v)), "y"]

df1 = pd.DataFrame.from_dict(module_dep1, orient='index')
note_str = tabulate(df1, headers=headers, tablefmt='psql')

# skip print
print(note_str)
# display(Markdown(note_str))

In [ ]:
def generate_per_node_depency_graph(graph: nx.DiGraph, path: str, png_gen: bool, reverse: bool=False) -> None:
	"""Generate node depency graph and anlaysis

	Parameters
	----------
	graph : nx.DiGraph
		dependency graph
	path : str
		path to save the graph
	png_gen : bool
		whether to generate png
	reverse : bool, optional
		whether to reverse the graph, by default False

	Returns
	-------
	None

	"""

	nodes = list(graph.nodes())
	# check dir and create if not exist
	if not os.path.exists(path):
		os.makedirs(path)

	# note_str = "## Module Dependency Graphs per modules"
	# display(Markdown(note_str))

	# png_gen = False
	for i, node in enumerate(nodes):
		filename = path + "/" + node + ".dot"
		if not reverse:
			T = nx.dfs_tree(graph, node)
		else:
			T = nx.bfs_tree(graph, node, reverse=reverse)
		if png_gen:
			pos = nx.nx_pydot.pydot_layout(T, prog='dot')
			# pos = nx.nx_agraph.graphviz_layout(graph, prog='dot')
			# nx.draw(graph, pos=pos, **options)
			# nx.write_dot(graph, 'module_dep.dot')
			nx.nx_pydot.write_dot(T, filename)
			os.system('dot -T png -O '+ filename)
		if not reverse:
			note_str = f"[{node}]({filename}.png): direct dependency {module_dep[node]}. All dependency including transitive: {len(T.nodes())-1}"
		else:
			note_str = f"[{node}]({filename}.png): All dependency including transitive: {len(T.nodes())}"
			# note_str = f"[{node}]({filename}.png): depends on {module_dep[node]}"
		display(Markdown(note_str))

In [ ]:
# test

T = nx.bfs_tree(graph, 'notification', reverse=True)
t = list(T.nodes())
len(t)

In [ ]:
# test

T = nx.bfs_tree(graph, 'sw_hu', reverse=True)
t2 = list(T.nodes())
len(t2)

In [ ]:
t3 = list(set(t) & set(t2))
len(t3)

## Module Dependency Graphs per modules

Click on the module name to see the dependency graph.

In [ ]:
path = "module_dots"
reverse = False
generate_per_node_depency_graph(graph, path, png_gen=png_gen, reverse=reverse)


## Reverse Module Dependency Graphs per modules

Click on the module name to see the dependency graph.

In [ ]:
path = "reverse_module_dots"
reverse = True
generate_per_node_depency_graph(graph, path, png_gen=png_gen, reverse=reverse)